# 日本の年齢別収入データ分析ガイド

このJupyterノートブックは、日本の公的統計データから年齢別収入データを取得・処理するためのサンプルコードです。複数の統計調査から年齢別の収入情報を取得し、将来の収入予測に使用できる形式に変換します。

## 概要

### 使用するデータソース

| 調査名 | 実施機関 | データの種類 | 年齢別情報 | 備考 |
|--------|----------|--------------|------------|------|
| 賃金構造基本統計調査 | 厚生労働省 | 賃金（所定内・外、賞与など） | ◎ あり | 最も詳細な個人ベースの賃金統計。年齢×学歴×勤続年数×職種など多変量分析可 |
| 家計調査 | 総務省統計局 | 世帯の実収入・支出 | ○ あり | 勤労者世帯の世帯主年齢別の平均月収がわかる。世帯単位のデータ |
| 全国家計構造調査（旧・全国消費実態調査） | 総務省統計局 | 所得・支出・貯蓄・負債等 | ○ あり | 5年ごと実施。世帯主年齢別に詳細な所得内訳がわかる |
| 民間給与実態統計調査 | 国税庁 | 年間給与（所得） | ◎ あり | 全国の給与所得者の年収実態。平均・中央値・分布あり。正規/非正規別も可 |
| 国民生活基礎調査 | 厚生労働省 | 世帯の所得・生活実態 | ○ あり | 世帯主年齢別の所得平均・中央値・分布あり。福祉・医療と併せた分析が可能 |
| 毎月勤労統計調査 | 厚生労働省 | 月次の給与・労働時間 | ○ 一部あり | 年報などで年齢別給与も公表。短期の変化や景気動向把握に向く |
| 労働力調査 | 総務省統計局 | 雇用状況（就業・失業） | ○（人数のみ） | 年齢別の就業者数・非労働力人口は把握できるが、賃金金額はなし |
| 就業構造基本調査 | 総務省統計局 | 職業・就業形態・月収階級等 | △（金額なし、階級） | 月収階級別の年齢分布あり。職種・雇用形態とのクロス集計が可能 |

## 環境セットアップ

### 必要なライブラリのインポート

In [1]:
import os
import sys
from pathlib import Path
import requests
import pandas as pd
from dotenv import load_dotenv

# 表示設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

### パッケージのパス設定とインポート

In [2]:
# パッケージのパスを追加（exampleフォルダから一つ上の階層のfpy_datareaderを参照）
sys.path.append(os.path.dirname(os.getcwd()))

from jpy_datareader.estat import MetaInfoReader, StatsDataReader
from fpy_datareader.future_income_estimater import (
    add_age_midpoint_column, 
    interpolate_age_income,
    create_age_income_dataframe_with_midpoint,
    process_family_income_expenditure_survey_data,
    process_family_income_consumption_wealth_survey_data,
    process_wage_structure_survey_data,
    process_wage_structure_survey_data_by_prefecture,
    process_living_conditions_survey_data,
    create_age_income_table
)

### APIキーの設定

In [4]:
# APIキーの読み込み
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")

**重要**: e-Stat APIを使用するには、事前にAPIキーの取得が必要です。[e-Stat API](https://www.e-stat.go.jp/api/)で登録してください。

## 1. 家計調査

家計調査は、総務省統計局が実施する世帯の収入・支出を調べる基幹統計です。

### データの特徴
- **対象**: 二人以上の世帯および単身世帯
- **頻度**: 月次調査
- **収入データ**: 世帯主の年齢階級別平均月収
- **用途**: 世帯単位での収入分析

### 利用可能なstatsDataId

#### 二人以上の世帯
- **月次データ**: `0002070010` - 用途分類（世帯主の年齢階級別）
- **年次データ**: `0002070011` - 用途分類（世帯主の年齢階級別）年次

#### 単身世帯（支出のみ）
- **月次データ**: `0003000795` - 用途分類（年齢階級別）
- **年次データ**: `0002190004` - 用途分類（年齢階級別）年次

**注意**: 単身世帯のデータ（statsDataId: `0003000795`, `0002190004`）は支出項目のみで、収入データは含まれていません。収入分析には二人以上世帯のデータ（statsDataId: `0002070010`, `0002070011`）を使用してください。

### メタデータの確認

In [5]:
# 二人以上世帯の年次データを例として使用
kakei_setai_statsDataId = "0002070011"
kakei_setai_metainfo = MetaInfoReader(
    api_key=appId, 
    statsDataId=kakei_setai_statsDataId,
)
kakei_setai_meta = kakei_setai_metainfo.read_class_objs()
kakei_setai_metainfo.OVERALL_TOTAL_NUMBER

322513

#### メタデータの構造確認

各メタデータの内容を順次確認します：

1. **表章項目** (`kakei_setai_meta[0]`): 金額、世帯数、人員などの測定項目
2. **用途分類** (`kakei_setai_meta[1]`): 収入・支出の詳細分類
3. **世帯区分** (`kakei_setai_meta[2]`): 二人以上世帯、勤労者世帯など
4. **年齢階級** (`kakei_setai_meta[3]`): 世帯主の年齢区分（25～29歳、30～34歳など）
5. **地域区分** (`kakei_setai_meta[4]`): 全国、地域別
6. **時間軸** (`kakei_setai_meta[5]`): 調査年月


In [6]:
kakei_setai_meta[0]

{'id': 'tab',
 'name': '表章項目',
 'meta_dataframe':   表章項目コード 表章項目  表章項目階層レベル
 0      01   金額       <NA>}

In [7]:
kakei_setai_meta[1]['meta_dataframe']

,用途分類コード,用途分類,用途分類階層レベル,用途分類単位,用途分類親コード
0,001,世帯数分布（抽出率調整）,1,一万分比,NaN
1,002,集計世帯数,1,世帯,NaN
2,003,世帯人員,1,人,NaN
3,004,18歳未満人員,2,人,003
4,005,65歳以上人員,2,人,003
...,...,...,...,...,...
260,261,一括払購入借入金純減率,3,％,251
261,262,財産純増率,3,％,251
262,263,エンゲル係数,1,％,NaN
263,264,年間収入,1,万円,NaN


In [8]:
kakei_setai_meta[2]

{'id': 'cat02',
 'name': '世帯区分（年次－二人以上の世帯）',
 'meta_dataframe':   世帯区分（年次－二人以上の世帯）コード                               世帯区分（年次－二人以上の世帯）  \
 0                  03                                二人以上の世帯（2000年～）   
 1                  04                        二人以上の世帯のうち勤労者世帯（2000年～）   
 2                  01          二人以上の世帯（農林漁家世帯を除く）（1985年～2007年,2017年）   
 3                  02  二人以上の世帯のうち勤労者世帯（農林漁家世帯を除く）（1985年～2007年,2017年）   
 
    世帯区分（年次－二人以上の世帯）階層レベル  
 0                      1  
 1                      1  
 2                      1  
 3                      1  }

In [9]:
kakei_setai_meta[3]

{'id': 'cat03',
 'name': '世帯主の年齢階級',
 'meta_dataframe':    世帯主の年齢階級コード 世帯主の年齢階級  世帯主の年齢階級階層レベル
 0          A00       平均              1
 1          425    24歳以下              1
 2          435    34歳以下              1
 3          205   25～29歳              1
 4          206   30～34歳              1
 5          207   35～39歳              1
 6          208   40～44歳              1
 7          209   45～49歳              1
 8          210   50～54歳              1
 9          211   55～59歳              1
 10         212   60～64歳              1
 11         213   65～69歳              1
 12         214   70～74歳              1
 13         215   75～79歳              1
 14         216   80～84歳              1
 15         570    70歳以上              1
 16         580    85歳以上              1
 17         565    65歳以上              1}

In [10]:
kakei_setai_meta[4]

{'id': 'area',
 'name': '地域区分',
 'meta_dataframe':   地域区分コード 地域区分  地域区分階層レベル
 0   00000   全国          1}

In [11]:
kakei_setai_meta[5]

{'id': 'time',
 'name': '時間軸（年次）',
 'meta_dataframe':     時間軸（年次）コード 時間軸（年次）  時間軸（年次）階層レベル
 0   1985000000   1985年             1
 1   1986000000   1986年             1
 2   1987000000   1987年             1
 3   1988000000   1988年             1
 4   1989000000   1989年             1
 5   1990000000   1990年             1
 6   1991000000   1991年             1
 7   1992000000   1992年             1
 8   1993000000   1993年             1
 9   1994000000   1994年             1
 10  1995000000   1995年             1
 11  1996000000   1996年             1
 12  1997000000   1997年             1
 13  1998000000   1998年             1
 14  1999000000   1999年             1
 15  2000000000   2000年             1
 16  2001000000   2001年             1
 17  2002000000   2002年             1
 18  2003000000   2003年             1
 19  2004000000   2004年             1
 20  2005000000   2005年             1
 21  2006000000   2006年             1
 22  2007000000   2007年             1
 23  2008000000   2008年            

### データ処理と結果

この関数は、家計調査データから年齢別収入カーブを作成し、以下の列を持つDataFrameを返します：
- `age`: 年齢（15歳から開始）
- `income`: 年収（円）
- `growth_rate`: 前年からの成長率
- `growth_rate_ma`: 移動平均された成長率

In [12]:
process_family_income_expenditure_survey_data(
    api_key=appId,
).head(30)

Successfully combined 1 DataFrames with total 11 rows
Successfully combined 1 DataFrames with total 11 rows


,age,income,growth_rate,growth_rate_ma
0,15,239772.580645,0.066667,0.062663
1,16,255757.419355,0.062500,0.062663
2,17,271742.258065,0.058824,0.058960
3,18,287727.096774,0.055556,0.055670
4,19,303711.935484,0.052632,0.052729
5,20,319696.774194,0.050000,0.050084
6,21,335681.612903,0.047619,0.047691
7,22,351666.451613,0.045455,0.045517
8,23,367651.290323,0.043478,0.043533
9,24,383636.129032,0.041667,0.041715



## 2. 全国家計構造調査

5年ごとに実施される詳細な家計調査で、より詳細な所得・資産情報を提供します。

### データの特徴
- **実施頻度**: 5年ごと
- **詳細度**: 所得・支出・貯蓄・負債の詳細分析が可能
- **最新データ**: 2019年調査結果

### 利用可能なstatsDataId

#### 主要なデータセット
- **基本データ**: `0003424621` - 世帯の種類・年齢階級・所得構成別データ
- 他にも以下のような詳細分類データが利用可能：
  - `0003426498` - 世帯の種類・世帯区分・性別・年齢階級・所得構成別
  - `0003426474` - 世帯の種類・企業規模・年齢階級・所得構成別
  - `0003426475` - 世帯人員・有業人員・年齢階級・所得構成別
  - `0003426476` - 世帯の種類・職業・年齢階級・所得構成別
  - `0003426478` - 世帯の種類・公的年金受給額階級・年齢階級・所得構成別
  - `0003426483` - 世帯の種類・年間収入階級・年齢階級・所得構成別
  - `0003426484` - 世帯の種類・年間収入十分位階級・年齢階級・所得構成別
  - `0003426485` - 世帯の種類・年間収入五分位階級・年齢階級・所得構成別

### メタデータの確認


In [13]:
ficw_statsDataId = "0003424621"
ficw_metainfo = MetaInfoReader(
    api_key=appId,
    statsDataId=ficw_statsDataId,
)
ficw_meta = ficw_metainfo.read_class_objs()
ficw_metainfo.OVERALL_TOTAL_NUMBER

473472


#### 重要なメタデータ項目

1. **世帯の種類**: 総世帯、二人以上の世帯、単身世帯
2. **世帯区分**: 全世帯、勤労者世帯、無職世帯、その他の世帯
3. **性別**: 平均、男、女
4. **収支項目分類**: 総数、平均、支払、実支出、消費支出など406項目
5. **年齢階級**: 30歳未満～85歳以上の32区分


In [14]:
ficw_meta[0]

{'id': 'tab',
 'name': '表章項目',
 'meta_dataframe':    表章項目コード              表章項目  表章項目階層レベル 表章項目単位
 0  06-2019         集計世帯数（概数）       <NA>    NaN
 1  07-2019             世帯数分布       <NA>    NaN
 2  08-2019              世帯人員       <NA>      人
 3  09-2019           18歳未満人員       <NA>      人
 4  10-2019           65歳以上人員       <NA>      人
 5  11-2019              有業人員       <NA>      人
 6  12-2019            世帯主の年齢       <NA>      歳
 7  01-2019  1世帯当たり1か月間の収入と支出       <NA>      円}

In [15]:
ficw_meta[1]

{'id': 'cat01',
 'name': '世帯の種類３区分',
 'meta_dataframe':   世帯の種類３区分コード 世帯の種類３区分  世帯の種類３区分階層レベル
 0           0      総世帯              1
 1           1  二人以上の世帯              1
 2           2     単身世帯              1}

In [16]:
ficw_meta[2]

{'id': 'cat02',
 'name': '世帯区分４区分',
 'meta_dataframe':   世帯区分４区分コード 世帯区分４区分  世帯区分４区分階層レベル
 0          0     全世帯             1
 1          1   勤労者世帯             1
 2          2    無職世帯             1
 3          3  その他の世帯             1}

In [17]:
ficw_meta[3]

{'id': 'cat03',
 'name': '世帯主の性別３区分',
 'meta_dataframe':   世帯主の性別３区分コード 世帯主の性別３区分  世帯主の性別３区分階層レベル
 0            0        平均               1
 1            1         男               1
 2            2         女               1}

### 可処分所得の確認

In [18]:
ficw_meta_df = ficw_meta[4]['meta_dataframe']
ficw_meta_df

,収支項目分類（細分類）406区分コード,収支項目分類（細分類）406区分,収支項目分類（細分類）406区分階層レベル,収支項目分類（細分類）406区分親コード
0,0,総数,1,NaN
1,1,平均,1,NaN
2,2,支払,1,NaN
3,21,実支出,2,2
4,2101,消費支出,3,21
...,...,...,...,...
401,5012,（特掲：用途分類）他の交際費,3,5
402,501201,（特掲：用途分類）つきあい費,4,5012
403,501202,（特掲：用途分類）住宅関係負担費,4,5012
404,501203,（特掲：用途分類）他の負担費,4,5012


In [19]:
# 可処分所得のコードを確認
ficw_meta_df = ficw_meta[4]['meta_dataframe']
disposable_income = ficw_meta_df[ficw_meta_df["収支項目分類（細分類）406区分コード"].str[0]=="4"]

,収支項目分類（細分類）406区分コード,収支項目分類（細分類）406区分,収支項目分類（細分類）406区分階層レベル,収支項目分類（細分類）406区分親コード
388,4,可処分所得,1,NaN


In [20]:
ficw_meta[5]

{'id': 'cat05',
 'name': '世帯主の年齢階級32区分',
 'meta_dataframe':    世帯主の年齢階級32区分コード   世帯主の年齢階級32区分  世帯主の年齢階級32区分階層レベル
 0              000             平均                  1
 1              101    30歳未満（５歳階級）                  1
 2              102    30～34（５歳階級）                  1
 3              103    35～39（５歳階級）                  1
 4              104    40～44（５歳階級）                  1
 5              105    45～49（５歳階級）                  1
 6              106    50～54（５歳階級）                  1
 7              107    55～59（５歳階級）                  1
 8              108    60～64（５歳階級）                  1
 9              109    65～69（５歳階級）                  1
 10             110    70～74（５歳階級）                  1
 11             111    75～79（５歳階級）                  1
 12             112    80～84（５歳階級）                  1
 13             113    85歳以上（５歳階級）                  1
 14             201  35歳未満（10歳階級1）                  1
 15             202  35～44（10歳階級1）                  1
 16             203  45

In [21]:
ficw_meta[6]

{'id': 'area',
 'name': '地域区分（全国）',
 'meta_dataframe':   地域区分（全国）コード 地域区分（全国）  地域区分（全国）階層レベル
 0       00000       全国              1}

In [22]:
ficw_meta[7]

{'id': 'time',
 'name': '時間軸(年次)',
 'meta_dataframe':    時間軸(年次)コード 時間軸(年次)  時間軸(年次)階層レベル
 0  2019000000   2019年             1}


### データ処理


In [23]:
process_family_income_consumption_wealth_survey_data(
    api_key=appId,
).head(30)

Successfully combined 1 DataFrames with total 32 rows


,age,income,growth_rate,growth_rate_ma
0,15,199477.090909,0.066667,0.062663
1,16,212775.563636,0.062500,0.062663
2,17,226074.036364,0.058824,0.058960
3,18,239372.509091,0.055556,0.055670
4,19,252670.981818,0.052632,0.052729
5,20,265969.454545,0.050000,0.050084
6,21,279267.927273,0.047619,0.047691
7,22,292566.400000,0.045455,0.045517
8,23,305864.872727,0.043478,0.043533
9,24,319163.345455,0.041667,0.041715



## 3. 賃金構造基本統計調査
https://www.e-stat.go.jp/stat-search/database?page=1&layout=datalist&toukei=00450091&tstat=000001011429&cycle=0&tclass1val=0&metadata=1&data=1
厚生労働省が実施する最も詳細な個人レベルの賃金統計です。

### データの特徴
- **対象**: 常用労働者（10人以上の事業所）
- **詳細度**: 年齢×学歴×勤続年数×職種の多変量分析が可能
- **データ項目**: 年齢、勤続年数、所定内・超過実労働時間数、現金給与額など

### 利用可能なstatsDataId

#### 全国データ
- **産業・年齢階級別**: `0003425893` - 一般_産業大・中分類_年齢階級別DB

#### 都道府県別データ
- **都道府県・年齢階級別**: `0003426933` - 一般_都道府県別_年齢階級別DB

### 全国データの分析


In [24]:
chingin_statsDataId = "0003425893"
chingin_metainfo = MetaInfoReader(
    api_key=appId, 
    statsDataId=chingin_statsDataId,
)
chingin_meta = chingin_metainfo.read_class_objs()
chingin_metainfo.OVERALL_TOTAL_NUMBER

3504384


#### メタデータの構造

1. **企業規模**: 10人以上、1,000人以上、100～999人、10～99人
2. **産業分類**: 109の詳細産業分類
3. **性別**: 男女計、男、女
4. **年齢階級**: ～19歳、20～24歳、...、70歳～の13区分
5. **学歴**: 中学、高校、専門学校、高専・短大、大学、大学院、不明
6. **民・公区分**: 民営事業所、民営+公営


In [25]:
chingin_meta[0]

{'id': 'tab',
 'name': '表章項目',
 'meta_dataframe':   表章項目コード           表章項目  表章項目階層レベル 表章項目単位
 0      33             年齢       <NA>      歳
 1      34           勤続年数       <NA>      年
 2      36      所定内実労働時間数       <NA>     時間
 3      38       超過実労働時間数       <NA>     時間
 4      40  きまって支給する現金給与額       <NA>     千円
 5      42         所定内給与額       <NA>     千円
 6      44   年間賞与その他特別給与額       <NA>     千円
 7      45           労働者数       <NA>     十人}

In [26]:
chingin_meta[1]

{'id': 'cat01',
 'name': '企業規模_基本',
 'meta_dataframe':   企業規模_基本コード       企業規模_基本  企業規模_基本階層レベル
 0         01  企業規模計（10人以上）             1
 1         02      1,000人以上             1
 2         03      100～999人             1
 3         04        10～99人             1}

In [27]:
chingin_meta[2]["meta_dataframe"]

,産業分類コード,産業分類,産業分類階層レベル
0,01,Ｔ１ 産業計,1
1,02,Ｃ 鉱業，採石業，砂利採取業,2
2,03,Ｄ 建設業,2
3,04,Ｄ０６ 総合工事業,3
4,05,Ｄ０７ 職別工事業（設備工事業を除く）,3
...,...,...,...
104,105,Ｒ９１ 職業紹介・労働者派遣業,3
105,106,Ｒ９２ その他の事業サービス業,3
106,107,Ｒ９３ 政治・経済・文化団体,3
107,108,Ｒ９４ 宗教,3


In [28]:
chingin_meta[3]

{'id': 'cat03',
 'name': '性別_基本',
 'meta_dataframe':   性別_基本コード 性別_基本  性別_基本階層レベル 性別_基本親コード
 0       01   男女計           1       NaN
 1       02     男           2        01
 2       03     女           2        01}

In [29]:
chingin_meta[4]

{'id': 'cat04',
 'name': '年齢階級_基本',
 'meta_dataframe':    年齢階級_基本コード 年齢階級_基本  年齢階級_基本階層レベル
 0          01     年齢計             1
 1          02    ～19歳             1
 2          03  20～24歳             1
 3          04  25～29歳             1
 4          05  30～34歳             1
 5          06  35～39歳             1
 6          07  40～44歳             1
 7          08  45～49歳             1
 8          09  50～54歳             1
 9          10  55～59歳             1
 10         11  60～64歳             1
 11         12  65～69歳             1
 12         13    70歳～             1}

In [30]:
chingin_meta[5]

{'id': 'cat05',
 'name': '学歴_基本８区分（2020年～）',
 'meta_dataframe':   学歴_基本８区分（2020年～）コード 学歴_基本８区分（2020年～）  学歴_基本８区分（2020年～）階層レベル  \
 0                  01              学歴計                      1   
 1                  02               中学                      2   
 2                  03               高校                      2   
 3                  04             専門学校                      2   
 4                  05            高専・短大                      2   
 5                  06               大学                      2   
 6                  07              大学院                      2   
 7                  08               不明                      2   
 
   学歴_基本８区分（2020年～）親コード  
 0                  NaN  
 1                   01  
 2                   01  
 3                   01  
 4                   01  
 5                   01  
 6                   01  
 7                   01  }

In [31]:
chingin_meta[6]

{'id': 'cat06',
 'name': '民・公区分',
 'meta_dataframe':   民・公区分コード  民・公区分  民・公区分階層レベル 民・公区分親コード
 0       02  民営事業所           2        01
 1       01  民営＋公営           1       NaN}

In [32]:
chingin_meta[7]

{'id': 'time',
 'name': '時間軸（2020～2023）',
 'meta_dataframe':   時間軸（2020～2023）コード 時間軸（2020～2023）  時間軸（2020～2023）階層レベル
 0        2023000000          2023年                    1
 1        2022000000          2022年                    1
 2        2021000000          2021年                    1
 3        2020000000          2020年                    1}

### データ処理

In [33]:
process_wage_structure_survey_data(
    api_key=appId
).head(30)

Successfully combined 1 DataFrames with total 13 rows
Successfully combined 1 DataFrames with total 13 rows
Successfully combined 1 DataFrames with total 13 rows


,age,income,growth_rate,growth_rate_ma
0,15,2.058455e+06,0.066667,0.062663
1,16,2.195686e+06,0.062500,0.062663
2,17,2.332916e+06,0.058824,0.058960
3,18,2.470147e+06,0.055556,0.061476
4,19,2.607377e+06,0.070050,0.064293
5,20,2.790023e+06,0.067273,0.066785
6,21,2.977715e+06,0.063032,0.063200
7,22,3.165408e+06,0.059295,0.058270
8,23,3.353100e+06,0.052482,0.053881
9,24,3.529078e+06,0.049865,0.049948


### 都道府県別データ

In [34]:
chingin_prefecture_statsDataId = "0003426933"
chingin_prefecture_metainfo = MetaInfoReader(
    api_key=appId, 
    statsDataId=chingin_prefecture_statsDataId,
)
chingin_prefecture_meta = chingin_prefecture_metainfo.read_class_objs()
chingin_prefecture_metainfo.OVERALL_TOTAL_NUMBER

8974368

都道府県別データには47都道府県の詳細な賃金情報が含まれています。

In [35]:
chingin_prefecture_meta[0]

{'id': 'tab',
 'name': '表章項目',
 'meta_dataframe':    表章項目コード           表章項目  表章項目階層レベル 表章項目単位
 0       01             年齢       <NA>      歳
 1       02           勤続年数       <NA>      年
 2       04      所定内実労働時間数       <NA>     時間
 3       06       超過実労働時間数       <NA>     時間
 4       08  きまって支給する現金給与額       <NA>     千円
 5       10         所定内給与額       <NA>     千円
 6       12   年間賞与その他特別給与額       <NA>     千円
 7       13           労働者数       <NA>     十人
 8       33             年齢       <NA>      歳
 9       34           勤続年数       <NA>      年
 10      36      所定内実労働時間数       <NA>     時間
 11      38       超過実労働時間数       <NA>     時間
 12      40  きまって支給する現金給与額       <NA>     千円
 13      42         所定内給与額       <NA>     千円
 14      44   年間賞与その他特別給与額       <NA>     千円
 15      45           労働者数       <NA>     十人}

In [36]:
chingin_prefecture_meta[1]

{'id': 'cat01',
 'name': '性別_基本',
 'meta_dataframe':   性別_基本コード 性別_基本  性別_基本階層レベル 性別_基本親コード
 0       01   男女計           1       NaN
 1       02     男           2        01
 2       03     女           2        01}

In [37]:
chingin_prefecture_meta[2]

{'id': 'cat02',
 'name': '年齢階級_基本',
 'meta_dataframe':    年齢階級_基本コード 年齢階級_基本  年齢階級_基本階層レベル
 0          01     年齢計             1
 1          02    ～19歳             1
 2          03  20～24歳             1
 3          04  25～29歳             1
 4          05  30～34歳             1
 5          06  35～39歳             1
 6          07  40～44歳             1
 7          08  45～49歳             1
 8          09  50～54歳             1
 9          10  55～59歳             1
 10         11  60～64歳             1
 11         12  65～69歳             1
 12         13    70歳～             1}

In [38]:
chingin_prefecture_meta[3]

{'id': 'cat03',
 'name': '企業規模_基本',
 'meta_dataframe':   企業規模_基本コード       企業規模_基本  企業規模_基本階層レベル
 0         01  企業規模計（10人以上）             1
 1         02      1,000人以上             1
 2         03      100～999人             1
 3         04        10～99人             1}

In [39]:
chingin_prefecture_meta[4]

{'id': 'cat04',
 'name': '産業分類',
 'meta_dataframe':    産業分類コード                     産業分類  産業分類階層レベル
 0       01                   Ｔ１ 産業計          1
 1       02           Ｃ 鉱業，採石業，砂利採取業          2
 2       03                    Ｄ 建設業          2
 3       07                    Ｅ 製造業          2
 4       08               Ｅ０９ 食料品製造業          3
 5       09         Ｅ１０ 飲料・たばこ・飼料製造業          3
 6       10                 Ｅ１１ 繊維工業          3
 7       11     Ｅ１２ 木材・木製品製造業(家具を除く)          3
 8       12            Ｅ１３ 家具・装備品製造業          3
 9       13        Ｅ１４ パルプ・紙・紙加工品製造業          3
 10      14              Ｅ１５ 印刷・同関連業          3
 11      15                 Ｅ１６ 化学工業          3
 12      17   Ｅ１８ プラスチック製品製造業(別掲を除く)          3
 13      18              Ｅ１９ ゴム製品製造業          3
 14      20           Ｅ２１ 窯業・土石製品製造業          3
 15      21                  Ｅ２２ 鉄鋼業          3
 16      22              Ｅ２３ 非鉄金属製造業          3
 17      23              Ｅ２４ 金属製品製造業          3
 18      24           Ｅ２５ はん用機械器具製造業 

In [40]:
chingin_prefecture_meta[5]

{'id': 'area',
 'name': '地域',
 'meta_dataframe':     地域コード    地域  地域階層レベル
 0   01000   北海道        2
 1   02000   青森県        2
 2   03000   岩手県        2
 3   04000   宮城県        2
 4   05000   秋田県        2
 5   06000   山形県        2
 6   07000   福島県        2
 7   08000   茨城県        2
 8   09000   栃木県        2
 9   10000   群馬県        2
 10  11000   埼玉県        2
 11  12000   千葉県        2
 12  13000   東京都        2
 13  14000  神奈川県        2
 14  15000   新潟県        2
 15  16000   富山県        2
 16  17000   石川県        2
 17  18000   福井県        2
 18  19000   山梨県        2
 19  20000   長野県        2
 20  21000   岐阜県        2
 21  22000   静岡県        2
 22  23000   愛知県        2
 23  24000   三重県        2
 24  25000   滋賀県        2
 25  26000   京都府        2
 26  27000   大阪府        2
 27  28000   兵庫県        2
 28  29000   奈良県        2
 29  30000  和歌山県        2
 30  31000   鳥取県        2
 31  32000   島根県        2
 32  33000   岡山県        2
 33  34000   広島県        2
 34  35000   山口県        2
 35  36000   徳島県

In [41]:
chingin_prefecture_meta[6]

{'id': 'time',
 'name': '時間軸（2020～2023）',
 'meta_dataframe':   時間軸（2020～2023）コード 時間軸（2020～2023）  時間軸（2020～2023）階層レベル
 0        2023000000          2023年                    1
 1        2022000000          2022年                    1
 2        2021000000          2021年                    1
 3        2020000000          2020年                    1}

### データ処理

In [42]:
process_wage_structure_survey_data_by_prefecture(
    api_key=appId
).head(30)

Successfully combined 1 DataFrames with total 13 rows
Successfully combined 1 DataFrames with total 13 rows
Successfully combined 1 DataFrames with total 13 rows


,age,income,growth_rate,growth_rate_ma
0,15,2.072120e+06,0.066667,0.062663
1,16,2.210262e+06,0.062500,0.062663
2,17,2.348403e+06,0.058824,0.058960
3,18,2.486545e+06,0.055556,0.065416
4,19,2.624686e+06,0.081870,0.072034
5,20,2.839569e+06,0.078678,0.077829
6,21,3.062979e+06,0.072939,0.073199
7,22,3.286390e+06,0.067980,0.066934
8,23,3.509800e+06,0.059883,0.061454
9,24,3.719978e+06,0.056500,0.056620



## 4. 国民生活基礎調査

厚生労働省が実施する世帯の所得・生活実態調査です。

### データの特徴
- **実施頻度**: 3年ごと（大規模調査）
- **特徴**: 福祉・医療データと組み合わせた分析が可能
- **所得データ**: 世帯主年齢別の平均・中央値・分布

### 利用可能なstatsDataId

#### 主要なデータセット
- **基本データ**: `0004021242` - 1世帯当たり平均所得金額・世帯人員1人当たり平均所得金額、世帯主の年齢（10歳階級）・年次別
- **詳細データ**: `0004021274` - 平均所得金額・平均世帯人員・平均有業人員、世帯主の年齢（10歳階級）別

### メタデータの確認


In [43]:
seikatsu_statsDataId = "0004021242"
seikatsu_metainfo = MetaInfoReader(
    api_key=appId, 
    statsDataId=seikatsu_statsDataId,
)
seikatsu_meta = seikatsu_metainfo.read_class_objs()
seikatsu_metainfo.OVERALL_TOTAL_NUMBER

#### 主要項目

1. **表章項目**: 1世帯当たり平均所得金額、世帯人員1人当たり平均所得金額
2. **年齢階級**: 29歳以下、30～39歳、40～49歳、50～59歳、60～69歳、70歳以上
3. **時系列データ**: 1985年から2022年までの長期データ

In [45]:
seikatsu_meta[0]

{'id': 'tab',
 'name': '表章項目',
 'meta_dataframe':   表章項目コード             表章項目  表章項目階層レベル 表章項目単位
 0      10     １世帯当たり平均所得金額       <NA>     万円
 1    1300  世帯人員１人当たり平均所得金額       <NA>     万円}

In [46]:
seikatsu_meta[1]

{'id': 'cat01',
 'name': '世帯主の年齢（１０歳階級）_102',
 'meta_dataframe':   世帯主の年齢（１０歳階級）_102コード 世帯主の年齢（１０歳階級）_102  世帯主の年齢（１０歳階級）_102階層レベル  \
 0                    1                総数                       1   
 1                    2             ２９歳以下                       2   
 2                    3            ３０～３９歳                       2   
 3                    4            ４０～４９歳                       2   
 4                    5            ５０～５９歳                       2   
 5                    6            ６０～６９歳                       2   
 6                    7             ７０歳以上                       2   
 7                    8         （再掲）６５歳以上                       2   
 8                    9         （再掲）７５歳以上                       2   
 
   世帯主の年齢（１０歳階級）_102親コード  
 0                   NaN  
 1                     1  
 2                     1  
 3                     1  
 4                     1  
 5                     1  
 6                     1  
 7                     1  
 

In [47]:
seikatsu_meta[2]

{'id': 'cat02',
 'name': '年次推移_2023',
 'meta_dataframe':    年次推移_2023コード    年次推移_2023  年次推移_2023階層レベル
 0           001  1985(昭和60)年               1
 1           002  1986(昭和61)年               1
 2           003  1987(昭和62)年               1
 3           004  1988(昭和63)年               1
 4           005   1989(平成元)年               1
 5           006   1990(平成2)年               1
 6           007   1991(平成3)年               1
 7           008   1992(平成4)年               1
 8           009   1993(平成5)年               1
 9           010   1994(平成6)年               1
 10          011   1995(平成7)年               1
 11          012   1996(平成8)年               1
 12          013   1997(平成9)年               1
 13          014  1998(平成10)年               1
 14          015  1999(平成11)年               1
 15          016  2000(平成12)年               1
 16          017  2001(平成13)年               1
 17          018  2002(平成14)年               1
 18          019  2003(平成15)年               1
 19          020  2004(平

In [48]:
seikatsu_meta[3]

{'id': 'time',
 'name': '調査年度',
 'meta_dataframe':   調査年度コード         調査年度  調査年度階層レベル
 0      16  2023年(令和5年)          1}

### データ処理

In [49]:
process_living_conditions_survey_data(
    api_key=appId
).head(30)

,age,income,growth_rate,growth_rate_ma
0,15,1.851818e+06,0.066667,0.062663
1,16,1.975273e+06,0.062500,0.062663
2,17,2.098727e+06,0.058824,0.058960
3,18,2.222182e+06,0.055556,0.055670
4,19,2.345636e+06,0.052632,0.052729
5,20,2.469091e+06,0.050000,0.050084
6,21,2.592545e+06,0.047619,0.047691
7,22,2.716000e+06,0.045455,0.045517
8,23,2.839455e+06,0.043478,0.043533
9,24,2.962909e+06,0.041667,0.041715



## 5. 民間給与実態統計調査

国税庁が実施する給与所得者の年収実態調査です。

### データの特徴
- **実施機関**: 国税庁
- **対象**: 全国の給与所得者
- **データ内容**: 年間給与（所得）の詳細分析
- **特徴**: 平均・中央値・分布データ、正規/非正規雇用別分析が可能

### 利用可能なstatsDataId

#### 主要なデータセット
- **全国計表 第10表**: `0004009621` - 事業所規模別及び年齢階層別の給与所得者数・給与額（1年を通じて勤務した給与所得者）（2014年～）
- **全国計表 第11表**: `0004009622` - 企業規模別及び年齢階層別の給与所得者数・給与額（1年を通じて勤務した給与所得者）（2014年～）
- **全国計表 第12表**: `0004009623` - 業種別及び年齢階層別の給与所得者数・給与額（1年を通じて勤務した給与所得者）（2014年～）
- **全国計表 第12表**: `0004009625` - 業種別及び年齢階層別の給与所得者数・給与額（1年を通じて勤務した給与所得者（乙欄適用者を除く））（2014年～）




## データの統合と活用

### 処理済みデータの特徴

各処理関数は、以下の共通フォーマットでデータを返します：

- **age**: 年齢（15歳から開始）
- **income**: 年収（円）
- **growth_rate**: 年収の成長率
- **growth_rate_ma**: 移動平均された成長率


## 注意事項

### メタデータの確認の重要性

**すべてのメタデータ確認処理は削除しないでください。** これらの処理は以下の理由で重要です：

1. **データ構造の理解**: 各調査の項目分類や階層構造を把握
2. **適切なパラメータ設定**: API呼び出し時の正しいコード指定
3. **データ品質の確保**: 期待するデータが取得できているかの確認
4. **再現性の保証**: 分析結果の再現と検証のための記録

### API使用時の注意

- e-Stat APIには利用制限があります
- 大量データ取得時は適切な間隔を空けてください
- APIキーは適切に管理してください

### データの解釈

- 各調査は対象範囲や調査手法が異なります
- 複数の調査結果を比較する際は、それぞれの特性を理解してください
- 時系列データを扱う際は、調査手法の変更や経済情勢の影響を考慮してください

## まとめ

このコードは、日本の公的統計データから年齢別収入データを体系的に取得・処理するためのツールです。複数の統計調査を組み合わせることで、より包括的で信頼性の高い年齢別収入分析が可能になります。